### Import Necessary Module and Set Path 

In [1]:
import sqlite3 as lite
import os
import sys
from pyne import nucname
sys.path.insert(0, '../scripts/')
jupyter_directory = os.getcwd()

In [2]:
import analysis as an

## Run Cyclus Input file

In [3]:
! rm cyclus.sqlite
! cyclus eg01-eg23-cycamore.xml

              :                                                               
          .CL:CC CC             _Q     _Q  _Q_Q    _Q    _Q              _Q   
        CC;CCCCCCCC:C;         /_\)   /_\)/_/\\)  /_\)  /_\)            /_\)  
        CCCCCCCCCCCCCl       __O|/O___O|/O_OO|/O__O|/O__O|/O____________O|/O__
     CCCCCCf     iCCCLCC     /////////////////////////////////////////////////
     iCCCt  ;;;;;.  CCCC                                                      
    CCCC  ;;;;;;;;;. CClL.                          c                         
   CCCC ,;;       ;;: CCCC  ;                   : CCCCi                       
    CCC ;;         ;;  CC   ;;:                CCC`   `C;                     
  lCCC ;;              CCCC  ;;;:             :CC .;;. C;   ;    :   ;  :;;   
  CCCC ;.              CCCC    ;;;,           CC ;    ; Ci  ;    :   ;  :  ;  
   iCC :;               CC       ;;;,        ;C ;       CC  ;    :   ; .      
  CCCi ;;               CCC        ;;;.      .C ;   

### The output file should be named 
         
          cyclus.sqlite

In [ ]:
filename = 'cyclus.sqlite'
# Wait for the complete message to move on
cur = an.get_cursor(filename)
# get simulation time info
init_year, init_month, duration, timestep = an.get_timesteps(cur)


# get fuel mass timeseries
fuel_dict = an.fuel_usage_timeseries(cur, ['uox', 'sfr_fuel'])
an.stacked_bar_chart(fuel_dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'Total Fuel Mass vs Time',
                  'total_fuel',
                  init_year)


# prints net capacity and number of reactor timeseries
an.plot_power(cur)

# get tailings mass timeseries
agent_ids = an.get_agent_ids(cur, 'enrichment')
tailings = an.facility_commodity_flux(cur, agent_ids,
                                      ['tailings'], True)
an.stacked_bar_chart(tailings, timestep,
                  'Year', 'Mass [MTHM]',
                  'Tailings vs Time',
                  'tailings', init_year)


# uranium utilization
dictionary = {}
dictionary['uranium_utilization'] = an.u_util_calc(cur)
an.stacked_bar_chart(dictionary, timestep,
                     'Years', 'U Utilization Factor',
                     'U Utilization vs Time',
                     'u_util', init_year)

# get plutonium flow
agent_ids = an.get_agent_ids(cur, 'separations')
combined = an.facility_commodity_flux(cur, agent_ids, ['sfr_Pu', 'uox_Pu'], True)
uox_Pu = an.facility_commodity_flux(cur, agent_ids, ['uox_Pu'], True)
mox_Pu = an.facility_commodity_flux(cur, agent_ids, ['sfr_Pu'], True)
an.stacked_bar_chart(combined, timestep, 'Years', 'Mass[MTHM]',
                  'Total reprocessed Pu outflux vs Time', 'combined',
                  init_year)
an.stacked_bar_chart(uox_Pu, timestep, 'Years', 'Mass[MTHM]',
                  'reprocessed Pu from spent UOX vs Time', 'uox_Pu',
                  init_year)
an.stacked_bar_chart(mox_Pu, timestep, 'Years', 'Mass[MTHM]',
                  'reprocessed Pu from spent MOX vs Time', 'sfr_Pu',
                  init_year)    

print('Finished!')

uox has not been used.
sfr_fuel has not been used.


In [5]:
# Number of Reactors Deployed
cur = an.get_cursor(filename)
lwr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "lwr"').fetchone()
print('The Number of LWRs Deployed:')
print(lwr[0])

sfr = cur.execute('SELECT count(*) FROM agententry '
                  'WHERE prototype = "fr"').fetchone()
print('The Number of FRs Deployed:')
print(sfr[0])

The Number of LWRs Deployed:
265
The Number of FRs Deployed:
2356


In [24]:
# Fresh and Spent Fuel Composition
cur = an.get_cursor(filename)

sfr_fuel_qualid = cur.execute('SELECT qualid FROM transactions '
                              'INNER JOIN resources ON '
                              'resources.resourceid = transactions.resourceid '
                              'WHERE commodity = "sfr_fuel"').fetchone()
fresh_sfr_mox_comp = cur.execute('SELECT nucid, massfrac FROM compositions '
                                 'WHERE qualid = ' + str(sfr_fuel_qualid[0])).fetchall()
print('Composition for fresh sfr fuel: ')
for isotope in fresh_sfr_mox_comp:
    print(nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %')

print('\n \n')

sfr_waste_comp = cur.execute('SELECT nucid, massfrac FROM recipes '
                             'LEFT OUTER JOIN compositions '
                             'ON recipes.qualid = compositions.qualid '
                             'WHERE recipe = "sfr_waste_recipe"').fetchall()
print('Composition for fresh sfr spent fuel: ')
for isotope in sfr_waste_comp:
    print(nucname.name(isotope[0]) + ': ' + str(isotope[1] * 100) + ' %')

Composition for fresh sfr fuel: 
U232: 7.641387132369223e-08 %
U233: 1.691727832793353e-07 %
U234: 0.013134963735120636 %
U235: 0.6309463459081436 %
U236: 0.4359981482910589 %
U238: 86.01992029647901 %
Pu238: 0.31638317090264156 %
Pu239: 6.583809648037809 %
Pu240: 3.1540637705938117 %
Pu241: 1.9080675065539083 %
Pu242: 0.9376448082728538 %
Pu244: 3.109563897755565e-05 %

 

Composition for fresh sfr spent fuel: 
H1: 6.768403861700439 %
H3: 1.9314999999983778e-05 %
He4: 0.0011680230123193587 %
Kr81: 8.733999999992664e-10 %
Kr85: 0.0032470999999972727 %
Sr90: 0.05990220999994968 %
Tc99: 0.1815299999998475 %
I129: 0.057136999999952 %
Cs134: 0.006368999999994649 %
Cs135: 0.3108399999997389 %
Cs137: 0.2763700399997679 %
Pb206: 1.0353499999991303e-16 %
Pb207: 1.6660922243959807e-14 %
Pb208: 6.474133140217591e-09 %
Pb210: 2.553670126997855e-15 %
Bi209: 3.1156750209278127e-12 %
Ra226: 4.086999999996567e-12 %
Ra228: 3.363099999997175e-19 %
Ac227: 4.331999999996361e-13 %
Th228: 1.100179999999075

### Plots

In [ ]:
# Display Net Capacity vs Time
from IPython.display import Image
Image(filename='power_plot.png')

# Display Number of Reactors vs Time
from IPython.display import Image
Image(filename='number_plot.png')

# Display Tailings vs Time
from IPython.display import Image
Image(filename='tailings.png')

# Display Total Fuel vs Time
from IPython.display import Image
Image(filename='total_fuel.png')

# Display Uranium Utilization vs Time
from IPython.display import Image
Image(filename='u_util.png')

# Display Separated Pu Flow
from IPython.display import Image
Image(filename='combined.png')

from IPython.display import Image
Image(filename='mox_Pu.png')

from IPython.display import Image
Image(filename='uox_Pu.png')